# Testing Assumptions
1. State Assumptions
2. Check Assumptions with figures using:
    * residuals
    * correlations
    * Number of modes

## Step 1: State Assumptions For Our Data
We have data X = {$X_i: i={1,...,n}, X_i ~ i.i.d. F$}

1. X_i are all identically and independently distributed in the distribution F.

All our data is i.i.d. So can check using a correlation matrix -> only checks for linear dependencies
-> can plot histogram of correlations to see if there are outliers

Plot scatter plots of each X_i with each other


paired scatter plots - plot subsamples

## Step 2: Check Assumptions

Ideas: Compute the comparison statistic for independence for "rows" closest to each other vs. farthest away
Read paper and compute the independence test